In [ ]:
import pandas as pd

In [ ]:
patient_info=pd.read_csv('AsthmaFiles/patient_info.csv')
patient_info

In [ ]:
patient_info[["user_key"]+list(patient_info)[-11:-2]]

In [ ]:
list_of_user_keys=[]
for x in patient_info[["user_key"]+list(patient_info)[-10:-2]].iterrows():
    if x[1]["pef_end_date"] - x[1]["pef_start_date"] >= 50 and x[1]["miband_end_date"]!="NaN":
        list_of_user_keys.append(x[1]["user_key"])
list_of_user_keys

In [ ]:
def get_peakflow_data(user_key):
    peakflow_data = pd.read_csv("AsthmaFiles/peakflow.csv")
    peakflow_data = peakflow_data[peakflow_data["user_key"]==user_key]
    peakflow_data = peakflow_data[["date","hour","pef_max"]]
    return peakflow_data


In [ ]:
def pair_weather(id,dates):
    weather = pd.read_csv("AsthmaFiles/environment.csv")
    for_id = weather.loc[weather['user_key'] == id]
    weather = for_id.loc[for_id['date'].isin(dates)]

    return weather

In [ ]:
peak_flow_data = get_peakflow_data(list_of_user_keys[0])
weather = pair_weather(list_of_user_keys[0],peak_flow_data["date"])


In [ ]:
def try_catch(row,default,peak_flow_data,weather,x):
    try:
        return weather.loc[weather['date'] == row["date"]].iloc[0][x]
    except IndexError:
        return default

In [ ]:
default = "NA"
for i in list_of_user_keys:
    peak_flow_data = get_peakflow_data(i)
    weather = pair_weather(i,peak_flow_data["date"])
    for x in list(weather.columns):
        if x not in peak_flow_data.columns:
            peak_flow_data[x] = peak_flow_data.apply(lambda row: try_catch(row,default,peak_flow_data,weather,x), axis = 1)
    peak_flow_data.to_csv(f"AsthmaFiles/{int(i)}.csv",index=False)